In [181]:
import pandas as pd # type: ignore

df = pd.read_csv(r'hcc_dataset.csv') #import data_frame
df


,Gender,Symptoms,Alcohol,HBsAg,HBeAg,HBcAb,HCVAb,Cirrhosis,Endemic,Smoking,...,ALP,TP,Creatinine,Nodules,Major_Dim,Dir_Bil,Iron,Sat,Ferritin,Class
0,Male,No,Yes,No,No,No,No,Yes,No,Yes,...,150,7.1,0.7,1,3.5,0.5,?,?,?,Lives
1,Female,?,No,No,No,No,Yes,Yes,?,?,...,?,?,?,1,1.8,?,?,?,?,Lives
2,Male,No,Yes,Yes,No,Yes,No,Yes,No,Yes,...,109,7,2.1,5,13,0.1,28,6,16,Lives
3,Male,Yes,Yes,No,No,No,No,Yes,No,Yes,...,174,8.1,1.11,2,15.7,0.2,?,?,?,Dies
4,Male,Yes,Yes,Yes,No,Yes,No,Yes,No,Yes,...,109,6.9,1.8,1,9,?,59,15,22,Lives
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Female,No,Yes,?,?,?,Yes,Yes,No,Yes,...,109,7.6,0.7,5,3,?,?,?,?,Lives
161,Female,Yes,No,?,?,?,?,Yes,No,No,...,280,6.7,0.7,1,2.2,2.3,?,?,?,Dies
162,Male,No,Yes,No,No,No,No,Yes,No,Yes,...,181,7.5,1.46,5,18.6,?,?,?,?,Lives
163,Male,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,...,170,8.4,0.74,5,18,?,?,?,?,Dies


In [182]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test = df.copy()

mapping = {"Yes": 1, "No": 0, "?": 2, "Male": 0, "Female": 1, "Lives": 1, "Dies": 0}


index=0
for column in df_test.columns:

    if df_test[column].dtype == 'object' and (index<23 or index==49)  :
        df_test[column] = df_test[column].map(mapping)
    index+=1
df_test

,Gender,Symptoms,Alcohol,HBsAg,HBeAg,HBcAb,HCVAb,Cirrhosis,Endemic,Smoking,...,ALP,TP,Creatinine,Nodules,Major_Dim,Dir_Bil,Iron,Sat,Ferritin,Class
0,0,0,1,0,0,0,0,1,0,1,...,150,7.1,0.7,1,3.5,0.5,?,?,?,1
1,1,2,0,0,0,0,1,1,2,2,...,?,?,?,1,1.8,?,?,?,?,1
2,0,0,1,1,0,1,0,1,0,1,...,109,7,2.1,5,13,0.1,28,6,16,1
3,0,1,1,0,0,0,0,1,0,1,...,174,8.1,1.11,2,15.7,0.2,?,?,?,0
4,0,1,1,1,0,1,0,1,0,1,...,109,6.9,1.8,1,9,?,59,15,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1,0,1,2,2,2,1,1,0,1,...,109,7.6,0.7,5,3,?,?,?,?,1
161,1,1,0,2,2,2,2,1,0,0,...,280,6.7,0.7,1,2.2,2.3,?,?,?,0
162,0,0,1,0,0,0,0,1,0,1,...,181,7.5,1.46,5,18.6,?,?,?,?,1
163,0,0,1,1,0,1,1,1,1,1,...,170,8.4,0.74,5,18,?,?,?,?,0


In [183]:
useless={}
index=0
#creating a list to see wich columns have the least yes's
for col in df_test.columns:
    if index>=23:break
    yes_count = (df_test[col] == 1).sum()
    if yes_count<=15: useless[col]=yes_count 
     
    index+=1
print(useless)


{'HBeAg': 1, 'Endemic': 10, 'Hemochro': 7, 'HIV': 3, 'NASH': 8}


In [184]:
""" #removing unecessary rows 
for index in useless.keys():
    df_test=df_test.drop(index,axis=1)
df_test.head() """

' #removing unecessary rows \nfor index in useless.keys():\n    df_test=df_test.drop(index,axis=1)\ndf_test.head() '

In [185]:
for col in df_test.columns:
    df_test[col]=label_encoder.fit_transform(df_test[col])
print(df_test['PS'])

0      0
1      0
2      1
3      0
4      0
      ..
160    0
161    1
162    0
163    1
164    0
Name: PS, Length: 165, dtype: int32


In [186]:
# ?=median  of the column 
#col = 29-50 not inclusive
df_test.replace('?', pd.NA, inplace=True)
df_test = df_test.apply(pd.to_numeric, errors='coerce')
index=29
for col in df_test.columns[29:49]:
    median = df_test[col].median(skipna=True)
    df_test[col].fillna(median,inplace=True)
for col in df_test.columns[23:25]:
    median = df_test[col].median(skipna=True)
    df_test[col].fillna(median,inplace=True)

df_test

,Gender,Symptoms,Alcohol,HBsAg,HBeAg,HBcAb,HCVAb,Cirrhosis,Endemic,Smoking,...,ALP,TP,Creatinine,Nodules,Major_Dim,Dir_Bil,Iron,Sat,Ferritin,Class
0,0,0,1,0,0,0,0,1,0,1,...,27,27,18,1,33,7,68,57,84,1
1,1,2,0,0,0,0,1,1,2,2,...,124,46,84,1,1,41,68,57,84,1
2,0,0,1,1,0,1,0,1,0,1,...,8,26,71,5,5,0,33,39,12,1
3,0,1,1,0,0,0,0,1,0,1,...,38,38,50,2,9,2,68,57,84,0
4,0,1,1,1,0,1,0,1,0,1,...,8,24,68,1,64,41,50,7,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1,0,1,2,2,2,1,1,0,1,...,8,32,18,5,28,41,68,57,84,1
161,1,1,0,2,2,2,2,1,0,0,...,68,22,18,1,19,28,68,57,84,0
162,0,0,1,0,0,0,0,1,0,1,...,43,31,62,5,15,41,68,57,84,1
163,0,0,1,1,0,1,1,1,1,1,...,35,41,22,5,14,41,68,57,84,0


In [187]:
from sklearn.model_selection import train_test_split
train_df_np=df_test.to_numpy()
X, y =train_df_np[:,:-1], train_df_np[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [188]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

model = DecisionTreeClassifier()

# Training the decision tree model
model.fit(X_train, y_train)


DecisionTreeClassifier()

In [189]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, predictions))

Accuracy: 0.5151515151515151
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.54      0.47        13
           1       0.62      0.50      0.56        20

    accuracy                           0.52        33
   macro avg       0.52      0.52      0.51        33
weighted avg       0.54      0.52      0.52        33

